# Parameters

In [1]:
batch_size = 32
dataset_name = 'plantVillage-test-mohanty'
dataset_path = '../datasets/' + dataset_name
model_diagram_path = 'vgg_model_diagram.png'
features_save_path = 'extracted-features.h5'

buffer_size = 1000

input_width = 224
input_height = 224

output_width = 7
output_height = 7
output_depth = 512


import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))



Classes:  ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
3


# Initialize model

In [2]:
import os, sys
# import pydot_ng as pydot
# pydot.find_graphviz()
# os.environ.get('PATH', '')

# remove these
# sys.path.append('..')
# remove these

from keras.utils.vis_utils import plot_model
from keras.applications import VGG16
from keras.optimizers import SGD

model = VGG16(weights="imagenet", include_top=False)

model.summary()
plot_model(model, to_file=model_diagram_path, show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])



Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
____________________________________________

# Load dataset

In [ ]:
import sys, os
sys.path.append('..')
import numpy as np
from imutils import paths
import random


from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# Get the list of image paths
image_paths = list(paths.list_images(dataset_path))
random.shuffle(image_paths)

#extract labels
labels = [p.split(os.path.sep)[-2] for p in image_paths]

# Encode labels and split dataset

In [3]:
from sklearn.preprocessing import LabelEncoder



# Convert labels in integers
le = LabelEncoder()
labels = le.fit_transform(labels)

NameError: name 'labels' is not defined

# Initialize Datsetwriter

In [ ]:
import utils
from utils.io.hdf5datasetwriter import HDF5DatasetWriter


dataset = HDF5DatasetWriter((len(image_paths), output_width * output_height * output_depth),
            features_save_path, dataKey="features", bufSize=buffer_size)

# Store the classes to to file
dataset.storeClassLabels(le.classes_)




# Forward propagate through model

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications import imagenet_utils

# loop over the image_paths in batches
for i in np.arange(0, len(image_paths), batch_size):
  # extract the image paths and labels for the current batch,
    batchPaths = image_paths[i:i + batch_size]
    batchLabels = labels[i:i + batch_size]
  # initializes a list to store the images about to be loaded and fed into VGG16.
    batchImages = []

    # loop over the images and labels in the current batch
    for (j, imagePath) in enumerate(batchPaths):

       # load the images and convert to correct size.
        image = load_img(imagePath, target_size=(input_width, input_height))
        # convert to numpy array
        image = img_to_array(image)

        # Expand dims and apply preprocessing
        # (1) expanding the dimensions
        # (2) subtracting the mean RGB pixel intensity from ImageNet dataset
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)

        # add the image to the batch
        batchImages.append(image)

  #  .vstack method used to “vertically stack”  images such that they have the shape
    # (N, 224, 224, 3) where N is the size of the batch.
    batchImages = np.vstack(batchImages)
    # pass the images through the network and use the outputs as our actual features
    features = model.predict(batchImages, batch_size=batch_size)

    # Output of VGG16's final Max Pool layer is (N, 512, 7, 7)
    # Flatten them so that they are of shape (N, 512 * 7 * 7)

    print(features.shape)
    features = features.reshape((features.shape[0], output_width * output_height * output_depth))

  # add the features and labels to our HDF5 dataset
    dataset.add(features, batchLabels)

dataset.close()

In [ ]:
>>> import sys
print(sys.version)